In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./OLID_2019/olid-training-v1.0.tsv", sep="\t")
df

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


### Configuration

In [3]:
## information we need to know from Config file
dataset_id = 1
label_columns = ["subtask_a", "subtask_b", "subtask_c"]
source_id = 1

def get_rowid(conn, table_name):
    '''Get the ID of the last inserted record in the specified table
    parameter conn: database connection'''
    c = conn.cursor()
    c.execute("SELECT MAX(rowid) FROM {}".format(table_name))
    last_inserted_id = c.fetchone()[0]
    return last_inserted_id

def create_dataset_id(conn):
    last_inserted_id = get_rowid(conn, "dataset")
    return last_inserted_id + 1

def get_text_source_id(conn, text_source):
    last_inserted_id = get_rowid(conn, "text_source")
    return last_inserted_id + 1

def get_language_id(conn, language):
    last_inserted_id = get_rowid(conn, "language")
    return last_inserted_id + 1

Create table ***Schema***

In [4]:
# Create a DataFrame with the specified columns
data = {
    "dataset_id": [dataset_id] * len(label_columns),  # Repeat dataset_id for each label column
    "label_name": label_columns
}

table_schema = pd.DataFrame(data)

def generate_temporary_schema(label_columns):
    data = {
        "dataset_id": [dataset_id] * len(label_columns),  # Repeat dataset_id for each label column
        "label_name": label_columns
    }
    table_schema = pd.DataFrame(data)
    return table_schema

def call_generate_temporary(table_name, *args):
    function_name = f'generate_temporary_{table_name}'
    if function_name in globals() and callable(globals()[function_name]):
        globals()[function_name](*args)
    else:
        print("Unsupported table name.")

In [5]:
table_schema

,dataset_id,label_name
0,1,subtask_a
1,1,subtask_b
2,1,subtask_c


In [6]:
df["dataset_id"] = dataset_id
df.rename(columns={'id': 'text_id',
                   'tweet':'text'
                   }, inplace=True)



In [7]:
df

,text_id,text,subtask_a,subtask_b,subtask_c,dataset_id
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,1
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,1
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,1
...,...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND,1
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN,1
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH,1
13238,27429,@USER Pussy,OFF,UNT,NaN,1


Create Table ***text***

In [8]:
table_text = df[['dataset_id', 'text_id', 'text']].drop_duplicates()

In [9]:
table_text['source_id'] = source_id

In [10]:
table_text

,dataset_id,text_id,text,source_id
0,1,86426,@USER She should ask a few native Americans wh...,1
1,1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1
2,1,16820,Amazon is investigating Chinese employees who ...,1
3,1,62688,"@USER Someone should'veTaken"" this piece of sh...",1
4,1,43605,@USER @USER Obama wanted liberals &amp; illega...,1
...,...,...,...,...
13235,1,95338,@USER Sometimes I get strong vibes from people...,1
13236,1,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,1
13237,1,82921,@USER And why report this garbage. We don't g...,1
13238,1,27429,@USER Pussy,1


Create Table ***label***

In [11]:
table_label = pd.DataFrame(columns = ['dataset_id', 'text_id', 'label_name', 'label_value'])

In [12]:
table_label

,dataset_id,text_id,label_name,label_value


In [13]:
for col in label_columns:
    label_value_pair = {
    'dataset_id': df['dataset_id'].values,
    'text_id': df['text_id'].values,
    'label_name': [col] * len(df),
    'label_value': df[col].values
    }
    temp_df =  pd.DataFrame(label_value_pair)
    table_label = pd.concat([table_label, temp_df], axis=0, ignore_index=True)


In [14]:
table_label

,dataset_id,text_id,label_name,label_value
0,1,86426,subtask_a,OFF
1,1,90194,subtask_a,OFF
2,1,16820,subtask_a,NOT
3,1,62688,subtask_a,OFF
4,1,43605,subtask_a,NOT
...,...,...,...,...
39715,1,95338,subtask_c,IND
39716,1,67210,subtask_c,NaN
39717,1,82921,subtask_c,OTH
39718,1,27429,subtask_c,NaN


### To-be-discussed

1. What to include in the Config JSON file, in which format?
2. How to handle the data source, also ask the user to include in the config file?
    - When coming from different sources?
3. label explaination
    - as a tuple in the config file (label_name, label_explaination), or?